# Loading CAO points information into a pandas data frame, using CAO website

http://www.cao.ie/index.php?page=points&p=2021

***

## INTRODUCTION

## WHAT IS CAO

### LOADING THE DATA

In [1]:
# HTTP request
import requests as rq
# Regular expressions
import re
# Dates and time
import datetime as dt
'''
# Data frames
import pandas as pd
# For downloading
import urllib.request as urlrq
'''

'\n# Data frames\nimport pandas as pd\n# For downloading\nimport urllib.request as urlrq\n'

## LEVEL 8 POINTS 2019, 2020, 2021

In [2]:
resp_2019 = rq.get ("https://web.archive.org/web/20191019135815/http://www2.cao.ie:80/points/l8.php")
resp_2020 = rq.get ("https://web.archive.org/web/20201108133105/http://www2.cao.ie/points/l8.php")
resp_2021 = rq.get ("http://www2.cao.ie/points/l8.php")
print (resp_2019)
print (resp_2020)
print (resp_2021)
#200 means ok

<Response [200]>
<Response [200]>
<Response [200]>


## SAVE THE ORIGINAL DATA

In [3]:
# Current date and time
now = dt.datetime.now()
# Format as a string
nowstr = (now.strftime("%Y%m%d_%H%M%S"))

In [4]:
# Create a file paths for the original data
path_2019 = ("data/cao2019_" +  nowstr + ".html")
path_2020 = ("data/cao2020_" +  nowstr + ".html")
path_2021 = ("data/cao2021_" +  nowstr + ".html")

In [5]:
# Fixing the wrong server encoding
# original_encoding = resp_2019.encoding
# Change to cp1252
resp_2019.encoding = "cp1252"
resp_2020.encoding = "cp1252"
resp_2021.encoding = "cp1252"

In [6]:
# Save the original html file
with open(path_2019, "w") as f:
    f.write(resp_2019.text)
with open(path_2020, "w") as f:
    f.write(resp_2020.text)
with open(path_2021, "w") as f:
    f.write(resp_2021.text)

## REGULAR EXPRESSIONS

In [7]:
re_course = re.compile(r"([A-Z]{2}[0-9]{3})(.*)")

## LOOP THROUGH THE LINES

In [8]:
path = ("data/cao2019_csv_" +  nowstr + ".csv")   
no_lines = 0
            
with open (path, "w") as f:
    # Loop
    for line in resp_2019.iter_lines():
        dline = (line.decode("cp1252"))
        # Match only lines with courses
        if re_course.fullmatch(dline):
            # Add one to the lines counter
            no_lines = no_lines + 1
            linesplit = re.split ("  +", dline)
            print (",".join(linesplit))
            f.write (",".join(linesplit) + "\n")     
print ("Total number of lines:", no_lines)

AL801,Software Design with Virtual Reality and Gaming,304,
AL802,Software Design with Cloud Computing,301,
AL803,Software Design with Mobile Apps and Connected Dev,309,
AL805,Network Management and Cloud Infrastructure,329,
AL810,Quantity Surveying,307,
AL820,Mechanical and Polymer Engineering,300,
AL830,General Nursing,410,
AL832,Psychiatric Nursing,387*,387,
AL836,Nutrition and Health Science,352,
AL837,Sports Science with Exercise Physiology,351,
AL838,Biotechnology,303,
AL839,Microbiology,322,
AL840,Pharmaceutical Sciences,328,
AL841,Athletic and Rehabilitation Therapy,423,
AL842,Bioveterinary Science,310,
AL843,Physical Activity and Health Science,310,
AL850,Business,307,
AL851,Business and Law,301,
AL852,Accounting (with optional placement),305,290,
AL853,Accounting and Law,305,
AL855,Hospitality Management (with International placeme,312,
AL857,Digital Marketing,299,
AL860,Social Care Practice,310,309,
AL861,Animation and Illustration (portfolio),#615,
AL863,Graphic and Digital 

In [9]:
path = ("data/cao2020_csv_" +  nowstr + ".csv")
    
no_lines = 0
            
with open (path, "w") as f:
    # Loop
    for line in resp_2020.iter_lines():
        dline = (line.decode("cp1252"))
        # Match only lines with courses
        if re_course.fullmatch(dline):
            # Add one to the lines counter
            no_lines = no_lines + 1
            linesplit = re.split ("  +", dline)
            print (",".join(linesplit))
            f.write (",".join(linesplit) + "\n")     
print ("Total number of lines:", no_lines)

AL801,Software Design with Virtual Reality and Gaming,303,
AL802,Software Design with Artificial Intelligence for C,332,
AL803,Software Design with Mobile Apps and Connected Dev,337,
AL805,Computer Engineering with Network Infrastructure,333,
AL810,Quantity Surveying,319,
AL811,Civil Engineering,320,
AL820,Mechanical and Polymer Engineering,308,
AL830,General Nursing,442,
AL832,Psychiatric Nursing,387*,384,
AL836,Nutrition and Health Science,334,
AL837,Sports Science with Exercise Physiology,351,
AL838,Biotechnology,307,
AL839,Microbiology,325,
AL840,Pharmaceutical Sciences,347,
AL841,Athletic and Rehabilitation Therapy,430,
AL842,Bioveterinary Science,313,
AL843,Physical Activity and Health Science,306,
AL850,Business,309,
AL851,Business and Law,295,
AL852,Accounting,309,
AL854,Sport Management (with international placement),302,299,
AL855,Hospitality Management (with international placeme,308,
AL857,Digital Marketing,304,
AL860,Social Care Practice,300,
AL861,Animation and Illustrati

In [10]:
path = ("data/cao2021_csv_" +  nowstr + ".csv")
    
no_lines = 0
            
with open (path, "w") as f:
    # Loop
    for line in resp_2021.iter_lines():
        dline = (line.decode("cp1252"))
        # Match only lines with courses
        if re_course.fullmatch(dline):
            # Add one to the lines counter
            no_lines = no_lines + 1
            linesplit = re.split ("  +", dline)
            print (",".join(linesplit))
            f.write (",".join(linesplit) + "\n")     
print ("Total number of lines:", no_lines)

AL801,Software Design for Virtual Reality and Gaming,300,
AL802,Software Design in Artificial Intelligence for Clo,313,
AL803,Software Design for Mobile Apps and Connected Devi,350,
AL805,Computer Engineering for Network Infrastructure,321,
AL810,Quantity Surveying,328,
AL811,Civil Engineering,
AL820,Mechanical and Polymer Engineering,327,
AL830,General Nursing,451*,444
AL832,Mental Health Nursing,440*,431
AL835,Pharmacology,356,
AL836,Nutrition and Health Science,346,
AL837,Sports Science with Exercise Physiology,357,
AL838,Biotechnology,324,
AL839,Microbiology,325,
AL840,Pharmaceutical Sciences,346,
AL841,Athletic and Rehabilitation Therapy,477,476*
AL842,Bioveterinary Science,338,
AL843,Physical Activity and Health Science,306,
AL849,Culinary Entrepreneurship,297,
AL850,Business,309,
AL851,Business and Law,302,
AL852,Accounting,336,
AL854,Sport Management (with international placement),300,299
AL855,Hospitality Management (with international placeme,309,
AL857,Digital Marketing,304,

## 2020 POINTS
http://www.cao.ie/index.php?page=points&p=2020

In [11]:
'''
# Create a file path for the original data
path = ("data/cao2020_" +  nowstr + ".xlsx")

urlrq.urlretrieve('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', path)

# Download and parse the excel spreadsheet
df = pd.read_excel("http://www2.cao.ie/points/CAOPointsCharts2020.xlsx", skiprows = 10)

df

# Spotcheck random row
df.iloc[753]

# Spotcheck last row
df.iloc[-1]

# Create a file path for the pandas data
path = ("data/cao2020_" +  nowstr + ".csv")

# Save pandas data frame to disk
df.to_csv(path)
'''

'\n# Create a file path for the original data\npath = ("data/cao2020_" +  nowstr + ".xlsx")\n\nurlrq.urlretrieve(\'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx\', path)\n\n# Download and parse the excel spreadsheet\ndf = pd.read_excel("http://www2.cao.ie/points/CAOPointsCharts2020.xlsx", skiprows = 10)\n\ndf\n\n# Spotcheck random row\ndf.iloc[753]\n\n# Spotcheck last row\ndf.iloc[-1]\n\n# Create a file path for the pandas data\npath = ("data/cao2020_" +  nowstr + ".csv")\n\n# Save pandas data frame to disk\ndf.to_csv(path)\n'

#### Regular expressions

### CAO DATA 2019, 2020, 2021

### DATA COMPARISON

# CONCLUSION

***

## REFERENCES

## LINKS TO CHECK